<a href="https://colab.research.google.com/github/evaie/evaie/blob/main/process_reperes_dame_de_nages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get references data
Find Mattieu's file and read it into a dataframe


In [15]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate the user
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Once authenticated, list the files that contains personal records for the atheletes


In [16]:
# Auto-iterate through all files that matches this query
#file_list = drive.ListFile({'q': "'root' in parents and title contains 'Champs' and trashed=false"}).GetList()
#sharedWithMe=true 
# id = 1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q
file_list = drive.ListFile({'q': "sharedWithMe=true and trashed=false and title contains '2021 - repères d'"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: 2021 - repères d'intensité, id: 1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q


once the id of the spreadsheet retrieved, load it. 
But first, we need to be allowed Gspread  / GSheet to access to our data.

In [24]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

Then load, the data into a dataframe.


In [30]:
sh = gc.open_by_key('1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q')
worksheet = sh.get_worksheet(0)
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

import pandas as pd
df_reperes = pd.DataFrame.from_records(rows)
df_reperes.describe()

,0,1,2,3,4,5,6,7,8,9,10
count,16,16,16,16,16,16,16,16,16,16,16
unique,15,14,6,15,3,15,15,15,2,15,14
top,,Louis,SHTC,,2000,,,,,,233
freq,2,2,4,2,13,2,2,2,15,2,2


Get the list of the files to parse

In [25]:
# files whose title contains SpdCoach and not in the trash
file_list = drive.ListFile({'q': "title contains 'SpdCoach' and trashed=false"}).GetList()
for ddd_file in file_list:
  # collect meta data from the filename
  filename = ddd_file['title']
  file_id = ddd_file['id']
  tokens = filename.split('-')
  rower = tokens[1].strip()
  training_type = tokens[2].split('.')[0].strip()
  date = tokens[0].split(' ')[2].strip() +':'+ tokens[0].split(' ')[3].strip()
  print('rower: %s, training type: %s, date: %s' % (rower, training_type, date))

rower: Adrien, training type: 500 et recup, date: 20210403:0814am
rower: Louis, training type: 500, date: 20210403:0814am
rower: Louis, training type: recup, date: 20210403:0824am
rower: Louis, training type: 1000, date: 20210331:0420pm
rower: Adrien, training type: 1000, date: 20210331:0420pm
rower: Yohan, training type: 1000, date: 20210331:0253pm


In [22]:
# this assume pyDrive has a token to read the GoogleDrive
myfile = drive.CreateFile({'id': '1B3u4YwBYpnV-k-ASEig01amQeoaWE7xg'})
myfile.GetContentFile('file.csv')
# then read it in a Dataframe
import pandas as pd
ddd = pd.read_csv('file.csv', skiprows=27, header=None)
ddd.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,Interval,Distance (GPS),Distance (IMP),Elapsed Time,Split (GPS),Speed (GPS),Split (IMP),Speed (IMP),Stroke Rate,Total Strokes,Distance/Stroke (GPS),Distance/Stroke (IMP),Heart Rate,Power,Catch,Slip,Finish,Wash,Force Avg,Work,Force Max,Max Force Angle,GPS Lat.,GPS Lon.
1,(Interval),(Meters),(Meters),(HH:MM:SS.tenths),(/500),(M/S),(/500),(M/S),(SPM),(Strokes),(Meters),(Meters),(BPM),(Watts),(Degrees),(Degrees),(Degrees),(Degrees),(Newtons),(Joules),(Newtons),(Degrees),(Degrees),(Degrees)
2,1,1.9,0.0,00:00:01.3,00:08:00.7,1.04,00:00:00.0,0.00,44.5,1,1.9,0.0,---,209,-41,2,30,7,243,550,659,-10,48.89648480,2.26630890
3,1,3.7,0.0,00:00:01.9,00:03:28.3,2.40,00:00:00.0,0.00,85.5,2,1.8,0.0,---,331,-38,2,31,8,358,501,648,-5,48.89649310,2.26633010
4,1,9.7,0.0,00:00:03.4,00:02:18.1,3.62,00:00:00.0,0.00,40.5,3,6.1,0.0,---,399,-44,1,31,9,369,563,653,-5,48.89652030,2.26640170
